<a href="https://colab.research.google.com/github/syoungk7/leetcode_mysolution/blob/main/Leetcode_sql_50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEETCODE SQL 50

## 1. Selcet

In [ ]:
-- 1757. Recyclable and Low Fat Products: find the ids of products that are both low fat and recyclable.

select product_id
from Products
where low_fats = 'Y' and recyclable = 'Y'

In [ ]:
-- 584. Find Customer Referee: Find the names of the customer that are not referred by the customer with id = 2.

select name
from Customer
where referee_id is null or referee_id != '2'

In [ ]:
-- 595. Big Countries: find the name, population, and area of the big countries.

select name, population, area
from World
where area >= 3000000 or population >= 25000000

In [ ]:
-- 1148. Article Views I: find all the authors that viewed at least one of their own articles.

select author_id as id
from Views
where author_id = viewer_id
group by author_id
order by id

--# distinct

In [ ]:
-- 1683. Invalid Tweets: find the IDs of the invalid tweets.

select tweet_id
from Tweets
Where char_length(content) > 12

--# length -> bytes

## 2. Basic Joins

In [ ]:
-- 1378. Replace Employee ID With The Unique Identifier: show the unique ID of each user, If a user does not have a unique ID replace just show null.

select u.unique_id, e.name
from Employees e
left join EmployeeUNI u on e.id = u.id

In [ ]:
-- 1068. Product Sales Analysis I: report the product_name, year, and price for each sale_id in the Sales table.

select p.product_name, s.year, s.price
from Sales s
right join Product p on s.product_id = p.product_id
where s.price is not Null

--# select p.product_name, s.year, s.price
--# from Sales s, Product p
--# where s.product_id = p.product_id

In [ ]:
-- 1581. Customer Who Visited but Did Not Make Any Transactions:
-- find the IDs of the users who visited without making any transactions and the number of times they made these types of visits.

select v.customer_id, count(v.customer_id) as count_no_trans
from Visits v
left join Transactions t on v.visit_id = t.visit_id
where t.amount is Null
group by v.customer_id

--# select customer_id, count(customer_id) as count_no_trans
--# from Visits
--# where visit_id not in (select visit_id from Transactions)
--# group by customer_id
--# order by count_no_trans Desc

In [ ]:
-- 197. Rising Temperature: find all dates'' Id with higher temperatures compared to its previous dates (yesterday).

select w.id
from Weather w, Weather e
where timestampdiff(day, e.recordDate, w.recordDate) = 1 and w.temperature - e.temperature > 0

--# With tmp_weather (id, recordDate, temperature, prev_day, prev_tem)
--# As (Select id, recordDate, temperature,
--#     Lag(recordDate) over (Order by recordDate) prev_day,
--#     Lag(temperature) over (Order by recordDate) prev_tem
--#     From Weather
--#     )
--# Select id
--# From tmp_weather
--# Where temperature - prev_tem > 0 and TIMESTAMPDIFF(Day, prev_day, recordDate) = 1

--# join
--# select w.id
--# from Weather w
--# Join Weather e on TIMESTAMPDIFF(Day, e.recordDate, w.recordDate) = 1 and w.temperature - e.temperature > 0

In [ ]:
-- 1661. Average Time of Process per Machine: find the average time each machine takes to complete a process.

select a.machine_id, round(sum((c.timestamp - a.timestamp))/count(a.process_id), 3) as processing_time
from Activity a
join Activity c on a.machine_id = c.machine_id and a.process_id = c.process_id and a.activity_type = 'start' and c.activity_type = 'end'
group by a.machine_id

In [ ]:
-- 577. Employee Bonus: report the name and bonus amount of each employee with a bonus less than 1000.

select e.name, b.bonus
from Employee e
left join Bonus b on e.empId = b.empId
where b.bonus < 1000 or b.bonus is Null

In [ ]:
-- 1280. Students and Examinations: find the number of times each student attended each exam.

select s.student_id, s.student_name, s.subject_name, count(e.student_id) as attended_exams
from (select student_id, student_name, subject_name
      from Students, Subjects) s
left join Examinations e on e.student_id = s.student_id and e.subject_name = s.subject_name
group by s.student_id, s.subject_name
order by s.student_id, s.subject_name

--# with s as (select student_id, student_name, subject_name from Students, Subjects)

In [ ]:
-- M 570. Managers with at Least 5 Direct Reports: find managers with at least five direct reports.

select e.name
from Employee e
join (select managerId
      from Employee
      group by managerId
      having count(managerId) >= 5) m on e.id = m.managerI

--# select name
--# from Employee
--# where id in (select managerId
--#              from Employee
--#              group by managerId
--#              having count(managerId) >= 5)

In [ ]:
-- M 1934. Confirmation Rate: find the confirmation rate of each user.

with noc as (select user_id, count(action) as num_of_confirmed
            from Confirmations
            where action = 'confirmed'
            group by user_id, action)
select s.user_id, ifnull(round(n.num_of_confirmed/count(c.user_id), 2), 0) as confirmation_rate
from Signups s
left join Confirmations c on s.user_id = c.user_id
left join noc n on c.user_id = n.user_id
group by s.user_id

## 3. Basic Aggregate Functions

In [ ]:
-- 620. Not Boring Movies: report the movies with an odd-numbered ID and a description that is not "boring".

SELECT *
FROM Cinema
WHERE id % 2 = 1 AND description != 'boring'
ORDER BY rating DESC

In [ ]:
-- 1251. Average Selling Price: find the average selling price for each product.

SELECT p.product_id,
    IFNULL(ROUND(SUM(p.price * u.units)/SUM(u.units), 2), 0) AS average_price
FROM Prices p
LEFT JOIN UnitsSold u ON p.product_id = u.product_id
    AND u.purchase_date BETWEEN start_date AND end_date
GROUP BY p.product_id

In [ ]:
-- 1075. Project Employees I: reports the average experience years of all the employees for each project, rounded to 2 digits.

SELECT p.project_id, ROUND(AVG(e.experience_years), 2) AS average_years
FROM Project p
LEFT JOIN Employee e on p.employee_id = e.employee_id
GROUP BY p.project_id

In [ ]:
-- 1633. Percentage of Users Attended a Contest: find the percentage of the users registered in each contest rounded to two decimals.

SELECT contest_id, ROUND(COUNT(contest_id)/(SELECT COUNT(DISTINCT user_id) FROM Users)*100, 2) AS percentage
FROM Register
GROUP BY contest_id
ORDER BY percentage DESC, contest_id

In [ ]:
-- 1211. Queries Quality and Percentage: find each query_name, the quality and poor_query_percentage.

SELECT query_name,
    ROUND(AVG(rating/position), 2) AS quality,
    ROUND(SUM(IF(rating < 3, 1, 0))/COUNT(query_name)*100, 2) AS poor_query_percentage
FROM Queries
GROUP BY query_name

In [ ]:
-- M 1193. Monthly Transactions I: find for each month and country, the number of transactions and their total amount,
-- the number of approved transactions and their total amount.

SELECT DATE_FORMAT(trans_date, '%Y-%m') AS month,
    country,
    COUNT(state) AS trans_count,
    SUM(IF(state = 'approved', 1, 0)) AS approved_count,
    SUM(amount) AS trans_total_amount,
    SUM(IF(state = 'approved', amount, 0)) AS approved_total_amount
FROM Transactions
GROUP BY month, country

In [ ]:
-- M 1174. Immediate Food Delivery II: find the percentage of immediate orders in the first orders of all customers, rounded to 2 decimal places.

SELECT ROUND(SUM(IF((d.order_date = d.customer_pref_delivery_date), 1, 0))/COUNT(d.customer_id)*100, 2) AS immediate_percentage
FROM Delivery d
JOIN (SELECT customer_id, MIN(order_date) AS min_date
      FROM Delivery
      GROUP BY customer_id) s ON d.order_date = s.min_date and d.customer_id = s.customer_id

In [ ]:
-- M 550. Game Play Analysis IV: report the fraction of players that logged in again on the day after the day they first logged in.

SELECT ROUND(COUNT(a.player_id)/(SELECT COUNT(DISTINCT player_id) FROM Activity), 2) AS fraction
FROM Activity a, (SELECT player_id, MIN(event_date) as min_date
                  FROM Activity
                  GROUP BY player_id) s
WHERE a.player_id = s.player_id and DATEDIFF(a.event_date, s.min_date) = 1

## 4. Sorting and Grouping

In [ ]:
-- 2356. Number of Unique Subjects Taught by Each Teacher: calculate the number of unique subjects each teacher teaches in the university.

SELECT teacher_id, count(DISTINCT subject_id) AS cnt
FROM Teacher
GROUP BY teacher_id

In [ ]:
-- 1141. User Activity for the Past 30 Days I: find the daily active user count for a period of 30 days ending 2019-07-27 inclusively.

SELECT activity_date AS day, COUNT(DISTINCT user_id) AS active_users
FROM Activity
--# WHERE DATEDIFF('2019-07-27', activity_date) BETWEEN 0 and 29
WHERE DATEDIFF('2019-07-27', activity_date) < 30 and activity_date < '2019-07-27'
GROUP BY activity_date

In [ ]:
-- M 1070. Product Sales Analysis III: select the product id, year, quantity, and price for the first year of every product sold.

SELECT product_id, year as first_year, quantity, price
FROM Sales
WHERE (product_id, year) in (SELECT product_id, MIN(year)
                             FROM Sales
                             GROUP BY product_id)

In [ ]:
-- 596. Classes More Than 5 Students: find all the classes that have at least five students.

SELECT class
FROM Courses
GROUP BY class
HAVING COUNT(student) >= 5

In [ ]:
-- 1729. Find Followers Count: Write a solution that will, for each user, return the number of followers.

SELECT user_id, COUNT(follower_id) AS followers_count
FROM Followers
GROUP BY user_id
ORDER BY user_id ASC

In [ ]:
-- 619. Biggest Single Number: Find the largest single number. If there is no single number, report null.

SELECT MAX(s.num) AS num
FROM (SELECT num
      FROM MyNumbers
      GROUP BY num
      HAVING COUNT(num) = 1) s

In [ ]:
-- M 1045. Customers Who Bought All Products: eport the customer ids from the Customer table that bought all the products in the Product table.

SELECT customer_id
FROM Customer
GROUP BY customer_id
HAVING COUNT(DISTINCT product_key) = (SELECT COUNT(product_key) FROM Product)

## 5. Sorting and Grouping

In [ ]:
-- 1731. The Number of Employees Which Report to Each Employee:  report the ids and the names of all managers,
-- the number of employees who report directly to them, and the average age of the reports rounded to the nearest integer.

SELECT e.employee_id, e.name, s.reports_count, s.average_age
FROM Employees e
JOIN (SELECT reports_to, COUNT(employee_id) AS reports_count, ROUND(AVG(age)) AS average_age
      FROM Employees
      GROUP BY reports_to
      HAVING reports_to is not null) s
      ON e.employee_id = s.reports_to
ORDER BY e.employee_id

In [ ]:
-- 1789. Primary Department for Each Employee: eport all the employees with their primary department.

(SELECT employee_id, department_id
FROM Employee
WHERE primary_flag = 'Y')
UNION
(SELECT employee_id, department_id
FROM Employee
GROUP BY employee_id
HAVING COUNT(department_id) = 1) --# employee belongs to only one department

In [ ]:
-- 610. Triangle Judgement: Report for every three line segments whether they can form a triangle.

SELECT x, y, z, IF((x + y > z and x + z > y and y + z > x), 'Yes', 'No') AS triangle
FROM Triangle

In [ ]:
-- M 180. Consecutive Numbers: Find all numbers that appear at least three times consecutively.

SELECT DISTINCT l.num AS ConsecutiveNums
FROM Logs l
JOIN Logs s ON l.id = s.id+1 and l.num = s.num
JOIN Logs s2 ON l.id = s2.id+2 and l.num = s2.num

In [ ]:
-- M 1164. Product Price at a Given Date: find the prices of all products on 2019-08-16.

SELECT p.product_id, IF(s.max_date is null, 10, new_price) AS price
FROM Products p
LEFT JOIN (SELECT product_id, MAX(change_date) as max_date
           FROM Products
           WHERE change_date <= '2019-08-16'
           GROUP BY product_id) s
        ON p.product_id = s.product_id
WHERE p.change_date = s.max_date or s.max_date is null
GROUP BY p.product_id

In [ ]:
-- M 1204. Last Person to Fit in the Bus: find the person_name of the last person that can fit on the bus without exceeding the weight limit.

SELECT person_name
FROM (SELECT person_name, SUM(weight) OVER(ORDER BY turn) AS total_weight
      FROM Queue
      ORDER BY total_weight DESC) s
WHERE total_weight <= 1000
LIMIT 1

In [ ]:
-- M 1907. Count Salary Categories: calculate the number of bank accounts for each salary category. The salary categories are:
-- "Low Salary": All the salaries strictly less than $20000.
-- "Average Salary": All the salaries in the inclusive range [$20000, $50000].
-- "High Salary": All the salaries strictly greater than $50000.

WITH selected AS (SELECT s.category, COUNT(s.category) AS accounts_count
                  FROM (SELECT account_id,
                            CASE
                                WHEN income < 20000 THEN 'Low Salary'
                                WHEN income <= 50000 THEN 'Average Salary'
                                ELSE 'High Salary'
                            END AS category
                        FROM Accounts) s
                GROUP BY s.category)

SELECT n.category, IFNULL(s.accounts_count, 0) AS accounts_count
FROM selected s
RIGHT JOIN (SELECT 'Low Salary' as category
            UNION
            SELECT 'Average Salary' as category
            UNION
            SELECT 'High Salary' as category) n
        ON n.category = s.category

## 6. Subqueries

In [ ]:
-- 1978. Employees Whose Manager Left the Company: Find the IDs of the employees whose salary < $30000 and whose manager left the company.

SELECT employee_id
FROM Employees
WHERE salary < 30000 and manager_id not in (SELECT employee_id FROM Employees)
ORDER BY employee_id

In [ ]:
-- M 626. Exchange Seats: swap the seat id of every two consecutive students.

SELECT s.id,
    CASE
        WHEN s.id%2=1 and a.id is Null THEN s.student
        WHEN s.id%2=1 THEN  a.student
        ELSE e.student
    END AS student
FROM Seat s
LEFT JOIN Seat e ON e.id = s.id - 1
LEFT JOIN Seat a ON a.id = s.id + 1

In [ ]:
-- M 1341. Movie Rating:
-- Find the name of the user who has rated the greatest number of movies. In case of a tie, return the lexicographically smaller user name.
-- Find the movie name with the highest average rating in February 2020. In case of a tie, return the lexicographically smaller movie name.

SELECT s.results
FROM ((SELECT u.name AS results, COUNT(m.movie_id) AS highest_count
       FROM MovieRating m, Users u
       WHERE m.user_id = u.user_id
       GROUP BY m.user_id
       ORDER BY highest_count DESC, u.name
       LIMIT 1)
      UNION
      (SELECT v.title AS results, AVG(m.rating) AS highest_avg
       FROM MovieRating m, Movies v
       WHERE m.created_at between '2020-02-01' and  '2020-02-29' and m.movie_id = v.movie_id
       GROUP BY m.movie_id
       ORDER BY highest_avg DESC, v.title
       LIMIT 1)) s

In [ ]:
-- M 1321. Restaurant Growth: Compute the moving average of how much the customer paid in a seven days window.

SELECT s.visited_on, s.amount, s.average_amount
FROM (SELECT visited_on,
        SUM(SUM(amount)) OVER (ORDER BY visited_on rows BETWEEN 6 preceding AND CURRENT ROW) AS amount,
        ROUND(AVG(SUM(amount)) OVER (ORDER BY visited_on rows BETWEEN 6 preceding AND CURRENT ROW), 2) AS average_amount
      FROM Customer
      GROUP BY visited_on) s
JOIN (SELECT c.visited_on
      FROM Customer c, Customer u
      WHERE DATEDIFF(c.visited_on, u.visited_on) = 6
      GROUP BY visited_on) c
    ON c.visited_on = s.visited_on
ORDER BY visited_on

In [ ]:
-- M 602. Friend Requests II: Who Has the Most Friends: find the people who have the most friends and the most friends number.

SELECT s.requester_id AS id, COUNT(s.requester_id) AS num
FROM  ((SELECT accepter_id AS requester_id, requester_id AS accepter_id, accept_date
        FROM RequestAccepted)
       UNION ALL
       (SELECT *
        FROM RequestAccepted)) s
GROUP BY s.requester_id
ORDER BY COUNT(s.requester_id) DESC
LIMIT 1

In [ ]:
-- M 585. Investments in 2016: report the sum of all total investment values in 2016 tiv_2016, for all policyholders who:
-- have the same tiv_2015 value as one or more other policyholders, and
-- are not located in the same city as any other policyholder (i.e., the (lat, lon) attribute pairs must be unique).

SELECT ROUND(SUM(tiv_2016), 2) AS tiv_2016
FROM Insurance
WHERE tiv_2015 in (SELECT tiv_2015
                   FROM Insurance
                   GROUP BY tiv_2015
                   HAVING COUNT(tiv_2015) != 1)
AND (lat, lon) in (SELECT lat, lon
                   FROM Insurance
                   GROUP BY lat, lon
                   HAVING COUNT(tiv_2015) = 1)

In [ ]:
-- H 185. Department Top Three Salaries: find the employees who are high earners in each of the departments.

WITH salary_order AS (
    SELECT d.name AS Department, e.name AS Employee, e.salary,
        DENSE_RANK () OVER (PARTITION BY d.id ORDER BY e.salary DESC) salary_rank
    FROM Employee e, Department d
    WHERE e.departmentId = d.id
    )

SELECT Department, Employee, salary
FROM salary_order
WHERE salary_rank <= 3

## 7. Advanced String Functions / Regex / Clause

In [ ]:
-- 1667. Fix Names in a Table:  fix the names so that only the first character is uppercase and the rest are lowercase.

SELECT user_id, CONCAT(UPPER(SUBSTRING(name, 1, 1)), LOWER(SUBSTRING(name, 2, LENGTH(name)))) AS name
FROM Users
ORDER BY user_id

--## LEFT and RIGHT
--# SELECT user_id, concat(upper(LEFT(name, 1)), lower(RIGHT(name, length(name)-1))) as name
--# FROM users
--# ORDER BY user_id;

In [ ]:
-- 1527. Patients With a Condition: find the patient_id, patient_name, and conditions of the patients who have Type I Diabetes.

SELECT patient_id, patient_name, conditions
FROM Patients
WHERE conditions LIKE 'DIAB1%' or conditions LIKE '% DIAB1%'

--# WHERE conditions REGEXP '\bDIAB1.*';

In [ ]:
-- 196. Delete Duplicate Emails: delete all duplicate emails, keeping only one unique email with the smallest id.

DELETE
FROM Person
WHERE (id, email) NOT IN (SELECT *
                          FROM (SELECT MIN(id), email
                                FROM Person
                                GROUP BY email
                                ) s )

--# Method 2
--# DELETE p1
--# FROM person p1, person p2
--# WHERE p1.Email = p2.Email AND p1.Id > p2.Id

In [ ]:
-- M 176. Second Highest Salary: find the second highest salary from the Employee table.

WITH selected AS (SELECT '2' as ranks, salary
                FROM Employee
                GROUP BY salary
                ORDER BY salary DESC
                LIMIT 1 OFFSET 1)

SELECT e.salary AS SecondHighestSalary
FROM selected e
RIGHT JOIN (SELECT '2' as ranks) s ON s.ranks = e.ranks

--## Method 2
--# SELECT IFNULL((SELECT DISTINCT salary  ## distinct!!
--#                 FROM Employee
--#                 GROUP BY salary
--#                 ORDER BY salary DESC
--#                 LIMIT 1 OFFSET 1), NULL) AS SecondHighestSalary

--## Method 3
--# SELECT MAX(Salary) AS SecondHighestSalary
--# FROM Employee
--# WHERE Salary < (SELECT MAX(Salary) FROM Employee)

In [ ]:
-- 1484. Group Sold Products By The Date: find for each date the number of different products sold and their names.

SELECT sell_date,
    COUNT(DISTINCT product) AS num_sold,
    GROUP_CONCAT(DISTINCT product ORDER BY product) AS products
FROM Activities
GROUP BY sell_date
ORDER BY sell_date

In [ ]:
-- 1327. List the Products Ordered in a Period: get the names of products that have at least 100 units ordered in February 2020 and their amount.

SELECT p.product_name, s.unit
FROM Products p, (SELECT product_id, SUM(unit) AS unit
                  FROM Orders
                  WHERE order_date BETWEEN '2020-02-01' and '2020-02-29'
                  # where left(order_date, 7) = "2020-02"
                  GROUP BY product_id
                  HAVING SUM(unit) >= 100
                 ) s
WHERE p.product_id = s.product_id

--# SELECT p.product_name AS product_name, sum(o.unit) AS unit FROM Products p
--# JOIN Orders o USING (product_id)
--# WHERE YEAR(o.order_date)='2020' AND MONTH(o.order_date)='02'
--# GROUP BY p.product_id
--# HAVING SUM(o.unit)>=100

In [ ]:
-- 1517. Find Users With Valid E-Mails: find the users who have valid emails.

Select *
from users
where REGEXP_LIKE(mail, '^[a-zA-Z][a-zA-Z0-9_.-]*@leetcode[.]com$')

--## '^[a-zA-Z][a-zA-Z0-9_.-]*\@leetcode\\.com$'